# Práctica 2

##### Autor: Pilar Navarro Ramírez

In [272]:
# Cargando el fichero version_information.py
# alojado el directorio de trabajo actual
# podremos comprobar las versiones concretas
# tanto del intérprete de Python, como de los
# módulos empleados.

%reload_ext version_information

%version_information numpy, matplotlib, sympy

Software versions
Python 3.5.2 64bit [GCC 5.4.0 20160609]
IPython 7.4.0
OS Linux 4.15.0 99 generic x86_64 with Ubuntu 16.04 xenial
numpy 1.16.3
matplotlib 3.0.3
sympy 1.5.1
Sun May 10 15:11:36 2020 CEST

## Ejercicios

1.- Obtenga mediante interpolación en el espacio $\mathbb{P}_2$ una fórmula para aproximar $f''(a)$ del tipo combinación
de $f(a-h)$, $f(a)$ y $f(a+h)$.

In [273]:
import numpy as np # Importamos el módulo NumPy con el pseudónimo np
import sympy as sp # Importamos el módulo SymPy con el pseudónimo sp

In [274]:
f = sp.Function('f')
a,h = sp.symbols('a,h')
z=sp.Symbol('z')
x = [a-h,a,a+h] #Nodos de interpolación
y = [f(x[0]),f(x[1]),f(x[2])]  # y los valores interpolados
D = sp.Symbol('D')   # D será la correspondiente diferencia dividida

In [275]:
p0 = y[0]  # vamos a ir construyendo el polinomio de interpolación
p1 = p0 + (z-x[0])/(x[1]-x[0])*(y[1]-y[0]) # usando la idea de Newton
p2 = p1 + (z-x[0])*(z-x[1])*D # D = f[x0,x1,x2]
p2  # ya tenemos la expresión genérica de dicho polinomio de grado 2

D*(-a + z)*(-a + h + z) + f(a - h) + (f(a) - f(a - h))*(-a + h + z)/h

In [276]:
#Comprobamos que la expresión obtenida interpola y_0 e y_1, 
p2.subs({z:x[0]})==y[0],p2.subs({z:x[1]})==y[1] 

(True, True)

In [277]:
# Garantizamos ahora que p(x2) = y2
sol2=sp.solve(p2.subs({z:x[2]})-y[2],D)  # para ello resolvemos esta ecuación y despejamos D
sol2 

[(-2*f(a) + f(a - h) + f(a + h))/(2*h**2)]

In [278]:
#Obtenemos ya el polinomio de interpolación buscado
D = sol2[0]
p2 = p1 + (z-x[0])*(z-x[1])*D
p2  

f(a - h) + (f(a) - f(a - h))*(-a + h + z)/h + (-a + z)*(-a + h + z)*(-2*f(a) + f(a - h) + f(a + h))/(2*h**2)

In [279]:
#Comprobamos que efectivamente p2 también interpola y_2
p2.subs({z:x[2]})==y[2] 

True

In [280]:
#Finalmente, derivamos dos veces el polinomio de interpolación encontrado para aproximar
#el valor de la derivada pedida
sp.diff(p2,z,2) #No hace falta evaluar la derivada en el punto x=a, al ser esta constant directamente.

(-2*f(a) + f(a - h) + f(a + h))/h**2

2.- Con la fórmula obtenida en el ejercicio 1, halle una tabla de aproximaciones y errores de $f_1''(2.5)$, siendo $f_1(x)=x^x$, para $h=10^{-i},\; i=1,\ldots,5.$

In [281]:
import numpy as np # Importamos el módulo NumPy con el pseudónimo np
import sympy as sp # Importamos el módulo SymPy con el pseudónimo sp

In [282]:
print(_)

(-2*f(a) + f(a - h) + f(a + h))/h**2


In [283]:
z = sp.Symbol('z')
h = sp.Symbol('h')

def f(z):
    return z**z

def aprox(z):
    return (-2*f(a) + f(a - h) + f(a + h))/h**2


In [284]:
a=2.5
# El valor exacto de la segunda derivada de la función f en a sería el siguiente
vexacto = (sp.diff(f(z),z,2)).subs({z:a}) 
vexacto

40.2416648563875

In [285]:
n=5 # Calculamos aproximaciones con la fórmula obtenida en el ejer1
dersnum = np.array([aprox(z).subs({h:10**(-i)}) for i in range(1,n+1)])
print(dersnum)  # con h desde 10^(-1) hasta el valor 10^(-n). 

[40.4205682979583 40.2434502309390 40.2416827078866 40.2416647560244
 40.2416411304784]


In [286]:
#Errores cometidos en la aproximación
errores = np.array(dersnum) - vexacto  
print(errores)                      

[0.178903441570853 0.00178537455153815 1.78514991731049e-5
 -1.00363052979446e-7 -2.37259090170028e-5]


Vemos que al principio, a medida que $h$ disminuye, el error también disminuye, pero a partir de $10^{-5}$ el error aumenta de nuevo. El error mínimo se alcanza para $h=10^{-4}$

3.- Sea $f_2(x)=\frac{x^2+40}{x+\sqrt{5x}+7}$. Calcule una tabla que recoja las derivadas de $f_2$ en $x_i=1,2,\ldots,10$, utilizando alguna de las fórmulas de derivación numérica de primer orden obtenidas al inicio de la práctica, con $h=10^{-3}$, y muestre al mismo tiempo el error cometido en cada punto. Repita el ejercicio con la fórmula centrada obtenida para la derivada primera y, finalmente, para la obtenida en el ejercicio 1 (con respecto a la segunda derivada).

In [287]:
import numpy as np # Importamos el módulo NumPy con el pseudónimo np
import sympy as sp # Importamos el módulo SymPy con el pseudónimo sp
import math

In [288]:
h = 10**(-3)
z = sp.Symbol('z')
a = sp.Symbol('a')

def f1(z):
    return (z**2+40)/(z+sp.sqrt(5*z)+7)

# Valor exacto de la derivada de la función f1 en xi para i=1..10
vexacto = np.array([sp.N(sp.diff(f1(z),z).subs({z:x})) for x in range(1,11)])
print("\nValores exactos de la derivada: \n", vexacto)


Valores exactos de la derivada: 
 [-0.633413841504903 -0.203729991363422 0.0135536765957583
 0.152356382446352 0.250865051903114 0.325234486346073 0.383753089267232
 0.431201820656649 0.470566739057635 0.503824070415537]


In [289]:
#Fórmula de derivación númerica no centrada con nodos [a,a+h] 
def aprox1(z):
    return (f1(z+h)-f1(z))/h

#Valores aproximados de la derivada usando la fórmula de derivación numérica
dersnum = np.array([aprox1(float(x)) for x in range(1,11)])
print("\nAproximaciones: \n", dersnum)

#Errores cometidos
errores = np.array(dersnum) - vexacto
print("\nErrores: \n", errores)


Aproximaciones: 
 [-0.633075850823062 -0.203588411025635 0.0136378345438892
 0.152413829637599 0.250907359192531 0.325267201964685 0.383779273533058
 0.431223324794061 0.470584759050041 0.503839418134255]

Errores: 
 [0.000337990681840927 0.000141580337787411 8.41579481309467e-5
 5.74471912467567e-5 4.23072894172871e-5 3.27156186126221e-5
 2.61842658260125e-5 2.15041374120317e-5 1.80199924059155e-5
 1.53477187179174e-5]


In [290]:
#Fórmula de derivación númerica centrada con tres nodos [a-h,a,a+h] 
def aprox2(z):
    return (-f1(z - h) + f1(z + h))/(2*h)

#Valores aproximados de la derivada usando la fórmula de derivación numérica
dersnum = np.array([aprox2(float(x)) for x in range(1,11)])
print("\nAproximaciones: \n", dersnum)

#Errores cometidos
errores = np.array(dersnum) - vexacto
print("\nErrores: \n", errores)


Aproximaciones: 
 [-0.633413983453845 -0.203730021215875 0.0135536643823819
 0.152356375979767 0.250865047976889 0.325234483748549 0.383753087445982
 0.431201819322435 0.470566738047573 0.503824069630454]

Errores: 
 [-1.41948942933290e-7 -2.98524533148825e-8 -1.22133763554022e-8
 -6.46658451786131e-9 -3.92622495626327e-9 -2.59752375164624e-9
 -1.82125048375781e-9 -1.33421368397890e-9 -1.01006214503485e-9
 -7.85083331678038e-10]


In [291]:
#Fórmula de derivación númerica obtenida en el ejercicio 1 

# Valor exacto de la derivada segunda de la función f1 en xi para i=1..10
vexacto = np.array([sp.N(sp.diff(f1(z),z,2).subs({z:x})) for x in range(1,11)])
print("\nValores exactos de la derivada segunda: \n", vexacto)

def aprox3(z):
    return (-2*f1(z) + f1(z - h) + f1(z + h))/h**2

#Valores aproximados de la derivada usando la fórmula de derivación numérica
dersnum = np.array([aprox3(float(x)) for x in range(1,11)])
print("\nAproximaciones: \n", dersnum)

#Errores cometidos
errores = np.array(dersnum) - vexacto
print("\nErrores: \n", errores)


Valores exactos de la derivada segunda: 
 [0.676265098285376 0.283220364176106 0.168340319928121 0.114907312895053
 0.0846224302869937 0.0654364313639429 0.0523721743690358
 0.0430109449028751 0.0360420057237485 0.0306970066620211]

Aproximaciones: 
 [0.676265261567721 0.283220380481453 0.168340323014604 0.114907315662549
 0.0846224312844868 0.0654364322727474 0.0523721741529926
 0.0430109432514314 0.0360420049361210 0.0306970076024982]

Errores: 
 [1.63282345200244e-7 1.63053464619090e-8 3.08648312463689e-9
 2.76749587135861e-9 9.97493074250677e-10 9.08804476118519e-10
 -2.16043224976481e-10 -1.65144369013159e-9 -7.87627567333526e-10
 9.40477102445714e-10]


Observamos que los errores cometidos son mucho menores con la fórmula centrada que con la no centrada, puesto que la centrada tiene un orden de exactitud mayor.

4.- Divida el intervalo $[1,2]$ en 100 partes iguales y aplique las fórmulas del rectángulo, Simpson y trapecio compuestas para aproximar la integral en dicho intervalo de $f_1$. Compare dichos resultados.


In [292]:
import numpy as np # Importamos el módulo NumPy con el pseudónimo np
import sympy as sp # Importamos el módulo SymPy con el pseudónimo sp
import scipy.integrate as sc # Importamos el módulo SciPy.integrate con el pseudónimo sc
import math

In [293]:
def formrectangizda(f,a,b,nx):
    """fórmula compuesta de los rectangulos a izquierda"""
    h = (b-a)/nx
    return h*sum([f(a+i*h) for i in range(0,nx)])

def formrectangdcha(f,a,b,nx):
    """fórmula compuesta de los rectangulos a la derecha"""
    h = (b-a)/nx
    return h*sum([f(a+(i+1)*h) for i in range(0,nx)])

def formtrapecios(f,a,b,nx):
    """fórmula compuesta de los trapecios"""
    h = (b-a)/nx
    return h/2*(f(a)+2*sum([f(a+i*h) for i in range(1,nx)])+f(b))

def Simpsoncompuesta(f,a,b,m):
    """fórmula de Simpson compuesta"""
    h= (b-a)/(2*m)
    P = sum([f(a+2*i*h) for i in range(1,m)])
    I = sum([f(a+(2*i-1)*h) for i in range(1,m+1)])
    E = f(a)+f(b)
    return h/3*(E+2*P+4*I)

In [294]:
a=1;b=2;n=100
z = sp.Symbol('z')

print("\nValor exacto : \n", sc.quad(lambda x:x**x,1,2)[0])
print("\nFórmula rectangulo izquierda compuesta : \n",formrectangizda(lambda z:z**z,a,b,n))
print("\nFórmula rectangulo derecha compuesta : \n",formrectangdcha(lambda z:z**z,a,b,n))
print("\nFórmula trapecio compuesta : \n",formtrapecios(lambda z:z**z,a,b,n))
print("\nFórmula Simpson compuesta : \n",Simpsoncompuesta(lambda z:z**z,a,b,int(n/2)))


Valor exacto : 
 2.050446234534731

Fórmula rectangulo izquierda compuesta : 
 2.0354943390855573

Fórmula rectangulo derecha compuesta : 
 2.065494339085557

Fórmula trapecio compuesta : 
 2.0504943390855574

Fórmula Simpson compuesta : 
 2.050446235955426


Observamos que las apoximaciones con las fórmulas compuestas del rectángulo izquierda y rectángulo derecha son las peores, seguidas por la fórmula compuesta del trapecio, que nos da una aproximación con 4 decimales exactos, siendo la mejor aproximación la obtenida con la fórmula de Simpson compuesta con la que obtenemos 8 decimales exactos en la aproximación.

5.- Repita el ejercicio 4 para $f_2$.

In [15]:
import numpy as np # Importamos el módulo NumPy con el pseudónimo np
import sympy as sp # Importamos el módulo SymPy con el pseudónimo sp
import math

In [16]:
def formrectangizda(f,a,b,nx):
    """fórmula compuesta de los rectangulos a izquierda"""
    h = (b-a)/nx
    return h*sum([f(a+i*h) for i in range(0,nx)])

def formrectangdcha(f,a,b,nx):
    """fórmula compuesta de los rectangulos a la derecha"""
    h = (b-a)/nx
    return h*sum([f(a+(i+1)*h) for i in range(0,nx)])

def formtrapecios(f,a,b,nx):
    """fórmula compuesta de los trapecios"""
    h = (b-a)/nx
    return h/2*(f(a)+2*sum([f(a+i*h) for i in range(1,nx)])+f(b))

def Simpsoncompuesta(f,a,b,m):
    """fórmula de Simpson compuesta"""
    h= (b-a)/(2*m)
    P = sum([f(a+2*i*h) for i in range(1,m)])
    I = sum([f(a+(2*i-1)*h) for i in range(1,m+1)])
    E = f(a)+f(b)
    return h/3*(E+2*P+4*I)

In [17]:
a=-1;b=1;n=2
z = sp.Symbol('z')
def f2(z):
    return abs(z)+1

print("\nValor exacto : \n", sp.N(sp.integrate(f2(z),[z,-1,1])))
print("\nFórmula rectangulo izquierda compuesta : \n",formrectangizda(f2,a,b,n))
print("\nFórmula rectangulo derecha compuesta : \n",formrectangdcha(f2,a,b,n))
print("\nFórmula trapecio compuesta : \n",sp.N(formtrapecios(f2,a,b,n)))
print("\nFórmula Simpson compuesta : \n", sp.N(Simpsoncompuesta(f2,a,b,int(n/2))))


Valor exacto : 
 3.00000000000000

Fórmula rectangulo izquierda compuesta : 
 3.0

Fórmula rectangulo derecha compuesta : 
 3.0

Fórmula trapecio compuesta : 
 3.00000000000000

Fórmula Simpson compuesta : 
 2.66666666666667


Podemos observar que todas las fórmulas nos dan un valor muy próximo a la solución exacta, siendo las aproximaciones por las fórmulas compuestas del rectángulo izquierda y rectángulo derecha un poco peores, pues sabemos que estas fórmulas tienen grado de exactitud 0. La mejor aproximación de todas es por la fórmula compuesta de Simpson, que tiene grado de exactitud 3, seguida por la del Trapecio, exacta en $\mathbb{P}_1$

6.- Sea $f_3(x)=x^{15} e^x$ en $[0,2]$. Vamos a dividir el intervalo en $10\times 2^n$ subintervalos, es decir, $10,\,20,\,40,\, 80,\ldots $ y a aplicar la fórmula de Simpson compuesta hasta que la diferencia entre dos aproximaciones consecutivas (por ejemplo, podrían
ser con $20$ y $40$ subintervalos) sea menor que $10^{-2}$, dando en tal caso por buena la última aproximación obtenida. Programe
y calcule dicha aproximación. Compare ambas aproximaciones con
el valor exacto.

In [298]:
import numpy as np # Importamos el módulo NumPy con el pseudónimo np
import sympy as sp # Importamos el módulo SymPy con el pseudónimo sp
import math

In [299]:
def Simpsoncompuesta(f,a,b,m):
    """fórmula de Simpson compuesta"""
    h= (b-a)/(2*m)
    P = sum([f(a+2*i*h) for i in range(1,m)])
    I = sum([f(a+(2*i-1)*h) for i in range(1,m+1)])
    E = f(a)+f(b)
    return h/3*(E+2*P+4*I)

z = sp.Symbol('z')

def f(z):
    return (z**15)*sp.exp(z)

vexacto=sp.N(sp.integrate(f(z),[z,a,b]))

In [300]:
a = 0 ; b = 2; i=2
nxx0=10 
xx0= np.linspace(a,b,nxx0)
nxx1=10*2
xx1 = np.linspace(a,b,nxx1)
aprox0 = sp.N(Simpsoncompuesta(f,a,b,nxx0))
aprox1 = sp.N(Simpsoncompuesta(f,a,b,nxx1))

while (abs(aprox0-aprox1)>=10**(-2)):    
    nxx=10*(2**i)
    xx = np.linspace(a,b,nxx)  
    aprox0 = aprox1
    aprox1 = sp.N(Simpsoncompuesta(f,a,b,nxx))
    i=i+1
    

print("\nValor exacto: \n",vexacto )
print("\nAproximaciones obtenidas: \nPenúltima aproximación - ",aprox0,"\nÚltima aproximación - ", aprox1)
print("\nErrores cometidos con las aproximaciones: \nPenúltima aproximación - ", abs(vexacto-aprox0), "\nÚltima aproximación - ", abs(vexacto-aprox1))


Valor exacto: 
 27062.5419875907

Aproximaciones obtenidas: 
Penúltima aproximación -  27062.7034855828 
Última aproximación -  27062.7024808912

Errores cometidos con las aproximaciones: 
Penúltima aproximación -  0.161497992103250 
Última aproximación -  0.160493300543749


7.- Calcule las fórmulas gaussianas con $2$ y $3$ nodos,en el intervalo $[-1,1]$, siendo la función peso el valor absoluto de la variable. Aplíquelas para aproximar la función $x\; e^x$ en $[-1,1]$ y compare los resultados con el valor exacto (organizando los cálculos de forma adecuada).

In [301]:
import numpy as np # Importamos el módulo NumPy con el pseudónimo np
import sympy as sp # Importamos el módulo SymPy con el pseudónimo sp
import math

In [302]:
a = -1 ; b = 1
x = sp.Symbol('x')
z = sp.Symbol('z')
g=sp.Function('g')

def w(x):
    return abs(x)

def f(x):
    return x*sp.exp(x)

vexacto=sp.N(sp.integrate(f(x),[x,a,b]))

###### Fórmula gaussiana con 2 nodos

In [303]:
n = 2
grexact = 2*n-1
p = sp.symbols('p0:'+ str(n)) # generamos una tupla de n nodos
nodos = list(p)
c = sp.symbols('c0:'+ str(n)) # generamos una tupla de n coeficientes
coefs = list(c)
incogs = coefs + nodos # lista completa de incógnitas
print("\nLista de incógnitas:\n",incogs)

ecs = [np.dot([(z**i).subs({z:nodos[j]}) for j in range(n)],coefs)-sp.integrate(w(x)*x**i,(x,a,b)) for i in range(grexact+1)]
print("\nEcuaciones a resolver:\n",ecs)

solsGauss = sp.solve(ecs,incogs)

for i in range(n):   
    coefs[i] = solsGauss[0][i]  
print("\nCoeficientes:\n",coefs)

for i in range(n):   # Y así le asignaríamos a los nodos
    nodos[i] = solsGauss[0][n+i] # las soluciones obtenidas
print("\nNodos:\n",nodos)

formGauss = np.dot([g(nodos[i]) for i in range(n)],coefs) 
print("\nFórmula gaussiana con 2 nodos:\n")
formGauss


Lista de incógnitas:
 [c0, c1, p0, p1]

Ecuaciones a resolver:
 [c0 + c1 - 1, c0*p0 + c1*p1, c0*p0**2 + c1*p1**2 - 1/2, c0*p0**3 + c1*p1**3]

Coeficientes:
 [1/2, 1/2]

Nodos:
 [-sqrt(2)/2, sqrt(2)/2]

Fórmula gaussiana con 2 nodos:



g(-sqrt(2)/2)/2 + g(sqrt(2)/2)/2

In [304]:
aprox = sp.N(np.dot([f(nodos[i]) for i in range(n)],coefs) )

print("\nValor exacto:\n",vexacto)
print("\nAproximación obtenida:\n",aprox)
print("\nError cometido:\n",abs(aprox-vexacto))


Valor exacto:
 0.735758882342885

Aproximación obtenida:
 0.542720820636303

Error cometido:
 0.193038061706581


###### Fórmula gaussiana con 3 nodos

In [305]:
n = 3
grexact = 2*n-1
p = sp.symbols('p0:'+ str(n)) # generamos una tupla de n nodos
nodos = list(p)
c = sp.symbols('c0:'+ str(n)) # generamos una tupla de n coeficientes
coefs = list(c)
incogs = coefs + nodos # lista completa de incógnitas
print("\nLista de incógnitas:\n",incogs)

ecs = [np.dot([(z**i).subs({z:nodos[j]}) for j in range(n)],coefs)-sp.integrate(w(x)*x**i,(x,a,b)) for i in range(grexact+1)]
print("\nEcuaciones a resolver:\n",ecs)

solsGauss = sp.solve(ecs,incogs)

for i in range(n):   
    coefs[i] = solsGauss[0][i]  
print("\nCoeficientes:\n",coefs)

for i in range(n):   # Y así le asignaríamos a los nodos
    nodos[i] = solsGauss[0][n+i] # las soluciones obtenidas
print("\nNodos:\n",nodos)

formGauss = np.dot([g(nodos[i]) for i in range(n)],coefs) 
print("\nFórmula gaussiana con 3 nodos:\n")
formGauss


Lista de incógnitas:
 [c0, c1, c2, p0, p1, p2]

Ecuaciones a resolver:
 [c0 + c1 + c2 - 1, c0*p0 + c1*p1 + c2*p2, c0*p0**2 + c1*p1**2 + c2*p2**2 - 1/2, c0*p0**3 + c1*p1**3 + c2*p2**3, c0*p0**4 + c1*p1**4 + c2*p2**4 - 1/3, c0*p0**5 + c1*p1**5 + c2*p2**5]

Coeficientes:
 [1/4, 3/8, 3/8]

Nodos:
 [0, -sqrt(6)/3, sqrt(6)/3]

Fórmula gaussiana con 3 nodos:



g(0)/4 + 3*g(-sqrt(6)/3)/8 + 3*g(sqrt(6)/3)/8

In [306]:
aprox = sp.N(np.dot([f(nodos[i]) for i in range(n)],coefs) ) 


print("\nValor exacto:\n",vexacto)
print("\nAproximación obtenida:\n",aprox)
print("\nError cometido:\n",abs(aprox-vexacto))


Valor exacto:
 0.735758882342885

Aproximación obtenida:
 0.557437075708894

Error cometido:
 0.178321806633991


8.- Programar las técnicas de integración de Romberg y adaptativa, para después aplicarlas a la aproximación de la siguiente integral $$\int_a^b p(x)\, dx$$
siendo  $\;a=\displaystyle\min_{0\leq i\leq 7}{d_i}$, $\;b=\displaystyle\max_{0\leq i\leq 7}{d_i}$ y 
$$p(x)=d_0 + d_1 x + d_2 x^2 + d_3 x^3+ d_4 x^4 + d_5 x^5 + d_6 x^6 + d_7 x^7 $$
(siendo $d_0, d_1, \ldots, d_7$ los dígitos de su DNI, pasaporte o tarjeta de residente).

In [307]:
import numpy as np # Importamos el módulo NumPy con el pseudónimo np
import sympy as sp # Importamos el módulo SymPy con el pseudónimo sp
import scipy.integrate as sc # Importamos el módulo SciPy.integrate con el pseudónimo sc
import math

In [308]:
def Romberg(f, a, b, tol):
    R = [[(b - a)*0.5* (f(a) + f(b))]]  # R[0][0]
    print(R[0])
    n = 1
    while True:
        h = float(b-a)/2**n
        R.append((n+1)*[None])  # Añadimos una línea vacía
        R[n][0] = 0.5*R[n-1][0] + h*sum(f(a+(2*i-1)*h) for i in range(1, 2**(n-1)+1)) 
        for k in range(1, n+1):
            R[n][k] = R[n][k-1] + (R[n][k-1] - R[n-1][k-1]) / (4**k - 1)
        print(R[n])
        if abs(R[n][n-1] - R[n][n]) < tol:
            return R[n][n]
        n += 1

In [309]:
def Adaptativa(f,a,b,total,tol):  
    m = (a+b)*0.5
    
    subintervalo1 = Simpsonsimple(a,f(a),f((a+m)*0.5),m,f(m))
    subintervalo2 = Simpsonsimple(m,f(m),f((m+b)*0.5),b,f(b))
    
    if( abs(total-subintervalo1-subintervalo2) < 10*tol):
        return subintervalo1 + subintervalo2
   
    return Adaptativa(f,a,m,subintervalo1,tol*0.5) + Adaptativa(f,m,b,subintervalo2,tol*0.5)        

In [310]:
x=sp.Symbol('x')  
a=2; b=9
m = (a+b)*0.5
tol=10**(-6)

def p(x):
    return 7+6*x+5*x**2+9*x**3+2*x**4+4*x**5+7*x**6+9*x**7

vexacto=sc.quad(p,a,b)[0]

In [311]:
print("\nValor exacto:\n", vexacto)
print("\nIntegración Romberg:")
Romberg(p, a, b,tol)


Valor exacto:
 53604180.175

Integración Romberg:
[164587584.0]
[87850414.12109375, 62271357.49479167]
[62648164.18804932, 54247414.210367836, 53712484.65807291]
[55896517.642258644, 53645968.79366175, 53605872.43254801, 53604180.175]
[54179241.975011125, 53606816.75259528, 53604206.61652418, 53604180.175, 53604180.175]


53604180.175

In [312]:
total = Simpsonsimple(a,f(a),f(m),b,f(b))
print("\nIntegración adaptativa:\n", Adaptativa(p,a,b,total,tol))


Integración adaptativa:
 53604180.17500022
